New model run 7th march

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SEBlock(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(in_channels, in_channels // reduction)
        self.fc2 = nn.Linear(in_channels // reduction, in_channels)

    def forward(self, x):
        batch, channels, _, _ = x.size()
        y = x.mean(dim=[2, 3])
        y = F.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y)).view(batch, channels, 1, 1)
        return x * y

class BottleneckBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1, expansion=4):
        super(BottleneckBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.GroupNorm(4, planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, groups=planes, bias=False)
        self.bn2 = nn.GroupNorm(4, planes)
        self.conv3 = nn.Conv2d(planes, planes * expansion, kernel_size=1, bias=False)
        self.bn3 = nn.GroupNorm(4, planes * expansion)
        self.se = SEBlock(planes * expansion)
        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * expansion, kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(4, planes * expansion)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out = self.se(out)
        out += self.shortcut(x)
        return F.relu(out)

class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.GroupNorm(4, 64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512 * 4, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * 4
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        return self.fc(out)

def ResNet18CIFAR():
    return ResNet(BottleneckBlock, [2, 2, 2, 2])

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import pickle
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
#from model import ResNet18CIFAR
import timm.optim.lookahead as Lookahead
from timm.optim.lookahead import Lookahead
# Custom Dataset Loader for CIFAR-10
class CIFARDataset(Dataset):
    def __init__(self, root_dir, train=True, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.data, self.labels = self._load_data(train)

    def _load_data(self, train):
        data_list, labels_list = [], []
        batches = [f"data_batch_{i}" for i in range(1, 6)] if train else ["test_batch"]
        for batch in batches:
            batch_path = os.path.join(self.root_dir, batch)
            with open(batch_path, 'rb') as f:
                batch_dict = pickle.load(f, encoding='bytes')
                data_list.append(batch_dict[b'data'])
                labels_list.extend(batch_dict[b'labels'])
        X = np.concatenate(data_list, axis=0)
        y = np.array(labels_list)
        return X, y

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx].reshape(3, 32, 32).transpose(1, 2, 0).astype("uint8")
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

def train():
    device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    print(f"Using device: {device}")

    transform_train = transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.AutoAugment(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])
    transform_val = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])

    root_dir = "/content/cifar-10-python/cifar-10-batches-py"
    trainset = CIFARDataset(root_dir=root_dir, train=True, transform=transform_train)
    valset = CIFARDataset(root_dir=root_dir, train=False, transform=transform_val)

    trainloader = DataLoader(trainset, batch_size=512, shuffle=True, num_workers=4)
    valloader = DataLoader(valset, batch_size=256, shuffle=False, num_workers=4)

    model = ResNet18CIFAR().to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    base_optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)
    optimizer = Lookahead(base_optimizer)  # Correct usage
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

    scaler = torch.cuda.amp.GradScaler()
    best_val_acc = 0.0

    for epoch in range(300):
        model.train()
        running_loss = 0.0
        for inputs, labels in tqdm(trainloader, desc=f"Epoch {epoch+1}/300"):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.cuda.amp.autocast():
                outputs = model(inputs)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item()
        scheduler.step()

        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in valloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_acc = 100.0 * correct / total
        print(f"Epoch [{epoch+1}/300] Loss: {running_loss/len(trainloader):.4f}, Val Acc: {val_acc:.2f}%")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print(f"Saved best model at epoch {epoch+1} with Val Acc: {val_acc:.2f}%")
    print("Training complete. Best Accuracy: {:.2f}%".format(best_val_acc))

if __name__ == "__main__":
    train()

Using device: cuda


<ipython-input-3-5db46cb0af60>:74: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1/300:   0%|          | 0/98 [00:00<?, ?it/s]<ipython-input-3-5db46cb0af60>:84: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1/300: 100%|██████████| 98/98 [00:16<00:00,  5.78it/s]


Epoch [1/300] Loss: 2.2517, Val Acc: 29.66%
Saved best model at epoch 1 with Val Acc: 29.66%


Epoch 2/300: 100%|██████████| 98/98 [00:14<00:00,  6.86it/s]


Epoch [2/300] Loss: 2.0606, Val Acc: 38.30%
Saved best model at epoch 2 with Val Acc: 38.30%


...

Epoch [298/300] Loss: 0.6061, Val Acc: 90.35%


Epoch 299/300: 100%|██████████| 98/98 [00:14<00:00,  6.82it/s]


Epoch [299/300] Loss: 0.6038, Val Acc: 90.30%


Epoch 300/300: 100%|██████████| 98/98 [00:14<00:00,  6.83it/s]


Epoch [300/300] Loss: 0.6039, Val Acc: 90.29%
Training complete. Best Accuracy: 90.59%


In [12]:
import torch
import torchvision.transforms as transforms
import pandas as pd
import pickle
import os
import numpy as np
#from model import ResNet18CIFAR
from torch.utils.data import DataLoader, Dataset

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

class CIFARTestDataset(Dataset):
    def __init__(self, data, ids, transform=None):
        """
        data: numpy array of shape (N, 32, 32, 3)
        ids: array-like of test image IDs
        transform: transforms to apply
        """
        self.data = data
        self.ids = ids
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx].astype("uint8")
        img_id = self.ids[idx]
        if self.transform:
            img = self.transform(img)
        return img, img_id

def inference():
    device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    print(f"Using device: {device}")

    model = ResNet18CIFAR().to(device)
    model.load_state_dict(torch.load("best_model.pth", map_location=device))
    model.eval()

    test_file = "/content/cifar_test_nolabel.pkl"
    test_dict = unpickle(test_file)
    print("Keys in test_dict:", test_dict.keys())

    test_images = test_dict[b'data'].reshape(-1, 32, 32, 3)  # Ensure correct shape
    test_ids = [str(i) for i in range(len(test_images))]  # Generate sequential IDs

    transform_test = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])

    test_dataset = CIFARTestDataset(test_images, test_ids, transform=transform_test)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    predictions = []
    image_ids = []

    with torch.no_grad():
        for images, ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Apply Softmax
            _, predicted = torch.max(probabilities, 1)
            predictions.extend(predicted.cpu().numpy().tolist())
            image_ids.extend(ids)

    submission = pd.DataFrame({"ID": image_ids, "Labels": predictions})
    submission.to_csv("submission.csv", index=False)
    print("Submission file saved as submission.csv")

if __name__ == "__main__":
    inference()

Using device: cuda
Keys in test_dict: dict_keys([b'data', b'ids'])


<ipython-input-12-8547a9c001a3>:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth", map_location=device))


Submission file saved as submission.csv


This model and training fetched us 0.77 on kaggle.

In [23]:
import pandas as pd

# Load and inspect submission file
submission_df = pd.read_csv("submission.csv")
print(submission_df.head())  # Show first few rows
print(submission_df["Labels"].value_counts())  # Show label distribution


   ID  Labels
0   0       6
1   1       1
2   2       8
3   3       6
4   4       9
Labels
3    1144
5    1116
8    1050
9    1049
1    1041
7    1022
2    1011
4     955
6     841
0     771
Name: count, dtype: int64


In [7]:
import torch
#from model import ResNet18CIFAR

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load model
model = ResNet18CIFAR().to(device)
checkpoint = torch.load("best_model.pth", map_location=device)
model.load_state_dict(checkpoint)
model.eval()

print("Model loaded successfully!")


Model loaded successfully!


<ipython-input-7-7049ceb695fc>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("best_model.pth", map_location=device)


New run (after the 0.766 run) (march 7th 8 pm)
This model and traiing gave 0.88 on kaggle

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Squeeze-and-Excitation Block
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(SEBlock, self).__init__()
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)

    def forward(self, x):
        b, c, _, _ = x.size()
        squeeze = x.view(b, c, -1).mean(dim=2)
        excitation = F.relu(self.fc1(squeeze))
        excitation = torch.sigmoid(self.fc2(excitation)).view(b, c, 1, 1)
        return x * excitation

# Residual Block with SE and GroupNorm
class ResBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.gn1 = nn.GroupNorm(8, out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.gn2 = nn.GroupNorm(8, out_channels)
        self.se = SEBlock(out_channels)

        self.downsample = None
        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.GroupNorm(8, out_channels)
            )

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = F.relu(self.gn1(out))
        out = self.conv2(out)
        out = self.gn2(out)
        out = self.se(out)

        if self.downsample:
            identity = self.downsample(x)

        return F.relu(out + identity)

# Optimized ResNet-34 for CIFAR-10
class ResNet34CIFAR(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet34CIFAR, self).__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False),
            nn.GroupNorm(8, 64),
            nn.ReLU()
        )
        self.layer1 = self._make_layer(64, 64, 3, stride=1)
        self.layer2 = self._make_layer(64, 128, 4, stride=2)
        self.layer3 = self._make_layer(128, 256, 6, stride=2)
        self.layer4 = self._make_layer(256, 512, 3, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = [ResBlock(in_channels, out_channels, stride)]
        for _ in range(1, num_blocks):
            layers.append(ResBlock(out_channels, out_channels, 1))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.fc(x)


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import DataLoader, Dataset
import pickle
import numpy as np
from timm.optim.lookahead import Lookahead
#from model import ResNet34CIFAR  # Import optimized model
from PIL import Image
# Function to load CIFAR-10 from local files
def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

# Custom dataset class
class CIFARDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data.reshape(-1, 3, 32, 32).astype("float32") / 255  # Normalize 0-1
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        img = self.data[idx] * 255  # Convert float32 (0-1) to uint8 (0-255)
        img = img.astype("uint8")   # Ensure data type is uint8
        img = Image.fromarray(img.transpose(1, 2, 0))  # Convert to PIL image

        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)  # Now the transform will work correctly

        return img, label

def load_cifar10_batches(root_dir):
    data_list, labels_list = [], []
    for i in range(1, 6):  # CIFAR-10 has 5 training batches
        batch_file = f"{root_dir}/data_batch_{i}"
        batch = unpickle(batch_file)
        data_list.append(batch[b'data'])
        labels_list.extend(batch[b'labels'])

    X_train = np.concatenate(data_list, axis=0)
    y_train = np.array(labels_list)
    return X_train, y_train

# Training function
def train():
    device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Load CIFAR-10 from local directory
    root_dir = "/content/cifar-10-python/cifar-10-batches-py"
    X_train, y_train = load_cifar10_batches(root_dir)

    # Data augmentation
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.AutoAugment(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])

    train_dataset = CIFARDataset(X_train, y_train, transform=transform_train)
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True, num_workers=4)

    # Initialize model
    model = ResNet34CIFAR().to(device)

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    base_optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=5e-4)
    optimizer = Lookahead(base_optimizer)  # Lookahead wraps AdamW
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=350)

    best_acc = 0.0

    # Training loop
    for epoch in range(350):
        model.train()
        running_loss = 0.0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()

        # Save best model
        if epoch % 50 == 0:
            torch.save(model.state_dict(), f"model_epoch_{epoch}.pth")

    torch.save(model.state_dict(), "best_model.pth")
    print("Training complete. Best model saved.")

if __name__ == "__main__":
    train()


Using device: cuda


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7b26682c76a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           ^^^^^^^^^^^Exception ignored in: ^<function _MultiProcessingDataLoaderIter.__del__ at 0x7b26682c76a0>^
^Traceback (most recent call last):
^  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1604, in __del__
^    ^self._shutdown_workers()^^
^  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 1587, in _shutdown_workers
^    ^if w.is_alive():^
^ ^  ^^ ^

Training complete. Best model saved.


In [22]:
import torch
import torchvision.transforms as transforms
import pandas as pd
import pickle
import os
import numpy as np
#from model import ResNet34CIFAR  # Import new model
from torch.utils.data import DataLoader, Dataset

def unpickle(file):
    with open(file, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
    return data_dict

class CIFARTestDataset(Dataset):
    def __init__(self, data, ids, transform=None):
        """
        data: numpy array of shape (N, 32, 32, 3)
        ids: array-like of test image IDs
        transform: transforms to apply
        """
        self.data = data
        self.ids = ids
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img = self.data[idx].astype("uint8")
        img_id = self.ids[idx]
        if self.transform:
            img = self.transform(img)
        return img, img_id

def inference():
    device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
    print(f"Using device: {device}")

    # Load optimized ResNet-34 model
    model = ResNet34CIFAR().to(device)
    model.load_state_dict(torch.load("/content/model_epoch_300.pth", map_location=device))
    model.eval()

    test_file = "/content/cifar_test_nolabel.pkl"
    test_dict = unpickle(test_file)
    print("Keys in test_dict:", test_dict.keys())

    test_images = test_dict[b'data'].reshape(-1, 32, 32, 3)  # Ensure correct shape
    test_ids = [str(i) for i in range(len(test_images))]  # Generate sequential IDs

    transform_test = transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
    ])

    test_dataset = CIFARTestDataset(test_images, test_ids, transform=transform_test)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

    predictions = []
    image_ids = []

    with torch.no_grad():
        for images, ids in test_loader:
            images = images.to(device)
            outputs = model(images)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)  # Apply Softmax
            _, predicted = torch.max(probabilities, 1)
            predictions.extend(predicted.cpu().numpy().tolist())
            image_ids.extend(ids)

    submission = pd.DataFrame({"ID": image_ids, "Labels": predictions})
    submission.to_csv("submission.csv", index=False)
    print("Submission file saved as submission.csv")

if __name__ == "__main__":
    inference()


Using device: cuda
Keys in test_dict: dict_keys([b'data', b'ids'])


<ipython-input-22-a463baec993b>:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/model_epoch_300.pth", map_location=device))


Submission file saved as submission.csv
